**Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
zip_path = "/content/drive/MyDrive/Dataset Datmin/Dataset Tubes-20251127.zip"

**Ekstrak ZIP ke folder**

In [ ]:
import zipfile
import os

extract_dir = "/content/dataset_tubes"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(os.listdir(extract_dir))

['A2022.csv', 'Gelombang (3).xlsx', '2022.csv', 'A2023.csv', 'A2021.csv', '2023.csv', '2021.csv', 'Gelombang (2).xlsx', 'Gelombang (4).xlsx', 'Gelombang (1).xlsx', 'Gelombang (5).xlsx', 'Gelombang (6).xlsx']


**Load Dataset 1 & Dataset 2 secara terpisah**

In [ ]:
import pandas as pd

# Dataset 1 (CSV)
dataset1_path = f"{extract_dir}/2021.csv"
dataset1 = pd.read_csv(dataset1_path)

# Dataset 2 (XLSX)
dataset2_path = f"{extract_dir}/Gelombang (1).xlsx"
dataset2 = pd.read_excel(dataset2_path)

dataset1.head(), dataset2.head()

(      KODE NAMA_PRODUK   UNIT   TANGGAL          NO_TRANSAKSI  QTY_MSK  \
 0  A000001  ANATON TAB  STRIP  06-07-21  1.13-210706.0908-003     10.0   
 1  A000001  ANATON TAB  STRIP  12-07-21   2.6-210712.1519-097      1.0   
 2  A000001  ANATON TAB  STRIP  12-07-21  2.11-210712.1633-013      1.0   
 3  A000001  ANATON TAB  STRIP  12-07-21  2.13-210712.1807-013      1.0   
 4  A000001  ANATON TAB  STRIP  12-07-21  2.11-210712.1855-018      1.0   
 
    NILAI_MSK  QTY_KLR  NILAI_KLR  
 0     2520.0        0          0  
 1     3000.0        0          0  
 2     3000.0        0          0  
 3     3000.0        0          0  
 4     3000.0        0          0  ,
   bandar agung_Andrean Syahrezi    Unnamed: 1   Unnamed: 2 Unnamed: 3  \
 0                     Location:  bandar agung          NaN        NaN   
 1                     Latitude:     -5.604619          NaN        NaN   
 2                    Longitude:    105.838853          NaN        NaN   
 3                 Time(UTC/GMT)   

**Import library**

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report

**Load Dataset 1 & Dataset 2**

In [ ]:
extract_dir = "/content/dataset_tubes"

# Dataset 1: transaksi tahun 2021
dataset1_path = f"{extract_dir}/2021.csv"
dataset1 = pd.read_csv(dataset1_path)

print("Dataset 1 (2021.csv):")
print(dataset1.head())
print(dataset1.info())

# Dataset 2: gelombang
dataset2_path = f"{extract_dir}/Gelombang (1).xlsx"
dataset2 = pd.read_excel(dataset2_path, header=3)

print("\nDataset 2 (Gelombang (1).xlsx):")
print(dataset2.head())
print(dataset2.info())

Dataset 1 (2021.csv):
      KODE NAMA_PRODUK   UNIT   TANGGAL          NO_TRANSAKSI  QTY_MSK  \
0  A000001  ANATON TAB  STRIP  06-07-21  1.13-210706.0908-003     10.0   
1  A000001  ANATON TAB  STRIP  12-07-21   2.6-210712.1519-097      1.0   
2  A000001  ANATON TAB  STRIP  12-07-21  2.11-210712.1633-013      1.0   
3  A000001  ANATON TAB  STRIP  12-07-21  2.13-210712.1807-013      1.0   
4  A000001  ANATON TAB  STRIP  12-07-21  2.11-210712.1855-018      1.0   

   NILAI_MSK  QTY_KLR  NILAI_KLR  
0     2520.0        0          0  
1     3000.0        0          0  
2     3000.0        0          0  
3     3000.0        0          0  
4     3000.0        0          0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138352 entries, 0 to 138351
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   KODE          138352 non-null  object 
 1   NAMA_PRODUK   138352 non-null  object 
 2   UNIT          138352 non-null  o

**Preprocessing Dataset 1 — Target = Jenis Transaksi (Masuk vs Keluar)**

In [ ]:
df1 = dataset1.copy()

# Asumsi: transaksi masuk punya QTY_MSK > 0, transaksi keluar punya QTY_KLR > 0
# Kita definisikan: 1 = Masuk, 0 = Keluar
df1['TIPE_TRANSAKSI'] = np.where(df1['QTY_MSK'] > 0, 1, 0)

print(df1[['QTY_MSK', 'QTY_KLR', 'TIPE_TRANSAKSI']].head())
print(df1['TIPE_TRANSAKSI'].value_counts())

   QTY_MSK  QTY_KLR  TIPE_TRANSAKSI
0     10.0        0               1
1      1.0        0               1
2      1.0        0               1
3      1.0        0               1
4      1.0        0               1
TIPE_TRANSAKSI
1    138352
Name: count, dtype: int64


**Preprocessing Dataset 2: Gelombang**

In [ ]:
# === LOAD DATASET GELOMBANG DENGAN HEADER BARIS KE-4 ===
dataset2_path = f"{extract_dir}/Gelombang (1).xlsx"

df2 = pd.read_excel(dataset2_path, header=4)
print("Kolom df2:", df2.columns)
print(df2.head())

# === TEMUKAN KOLOM HSIG ===
hsig_col = [c for c in df2.columns if "hsig" in c.lower()][0]
print("Kolom Hsig:", hsig_col)

# === BERSIHKAN BARIS KOSONG ===
df2 = df2.dropna(subset=[hsig_col])
df2[hsig_col] = pd.to_numeric(df2[hsig_col], errors='coerce')
df2 = df2.dropna(subset=[hsig_col])

# === HITUNG MEDIAN GT DAN BUAT KELAS GELOMBANG ===
median_hsig = df2[hsig_col].median()
print("Median Hsig:", median_hsig)

df2['WAVE_CLASS'] = np.where(df2[hsig_col] > median_hsig, 1, 0)
print(df2[[hsig_col, 'WAVE_CLASS']].head())
print(df2['WAVE_CLASS'].value_counts())

Kolom df2: Index(['Time(UTC/GMT)', 'Hsig(m)', 'Hsig(Scale)', 'Hmax(m)', 'Hmax(Scale)',
       'WaveDir(deg)', 'WaveDir(compass)', 'PrimSwell(m)', 'PrimSwell(Scale)',
       'PrimSwellDir(deg)', 'PrimSwellDir(compass)', 'WindSea(m)',
       'WindSea(Scale)', 'WindSeaDir(deg)', 'WindSeaDir(compass)',
       'WavePeriod(s)', 'SurfCurrentSpd(cm/s)', 'SurfCurrentDir',
       'SurfCurrentDir(compass)', 'SeaSurfaceTemperature(°C)',
       'SeaSurfaceSalinity(PSU)', 'WindSpeed(knots)', 'WindDir(deg)',
       'WindDir(compass)'],
      dtype='object')
        Time(UTC/GMT)  Hsig(m) Hsig(Scale)  Hmax(m) Hmax(Scale)  WaveDir(deg)  \
0 2024-07-29 00:00:00  0.08409      Smooth  0.16817      Smooth           334   
1 2024-07-29 01:00:00  0.08298      Smooth  0.16596      Smooth           334   
2 2024-07-29 02:00:00  0.08188      Smooth  0.16375      Smooth           334   
3 2024-07-29 03:00:00  0.08077      Smooth  0.16154      Smooth           334   
4 2024-07-29 04:00:00  0.07966      Smooth  0.

**Eksekusi Model & Hasil**

In [ ]:
def run_experiments(df, target, nama):
    print("\n", nama, "\n")

    # Fitur & label
    X = df.drop(columns=[target])
    y = df[target]

    # Deteksi tipe kolom
    cat_cols = X.select_dtypes(include=['object','category']).columns
    num_cols = X.select_dtypes(include=['int64','float64']).columns

    print("Kategorikal :", list(cat_cols))
    print("Numerik     :", list(num_cols))

    # Preprocessing (encode + scale)
    prep = ColumnTransformer([
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols),
        ('num', StandardScaler(), num_cols)
    ])

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )

    # 1. BASELINE
    model_base = Pipeline([
        ('prep', prep),
        ('tree', DecisionTreeClassifier(random_state=42))
    ])
    model_base.fit(X_train, y_train)
    pred_base = model_base.predict(X_test)

    print("\n[BASELINE]")
    print("Accuracy :", accuracy_score(y_test, pred_base))
    print(classification_report(y_test, pred_base))

    # 2. FEATURE SELECTION
    model_fs = Pipeline([
        ('prep', prep),
        ('fs', SelectFromModel(RandomForestClassifier(
            n_estimators=200, random_state=42, n_jobs=-1
        ))),
        ('tree', DecisionTreeClassifier(random_state=42))
    ])
    model_fs.fit(X_train, y_train)
    pred_fs = model_fs.predict(X_test)

    print("\n[FEATURE SELECTION]")
    print("Accuracy :", accuracy_score(y_test, pred_fs))
    print(classification_report(y_test, pred_fs))

    # 3. PCA ONLY
    model_pca = Pipeline([
        ('prep', prep),
        ('pca', PCA(n_components=5)),
        ('tree', DecisionTreeClassifier(random_state=42))
    ])
    model_pca.fit(X_train, y_train)
    pred_pca = model_pca.predict(X_test)

    print("\n[PCA ONLY]")
    print("Accuracy :", accuracy_score(y_test, pred_pca))
    print(classification_report(y_test, pred_pca))

    # 4. FEATURE SELECTION + PCA
    model_fspca = Pipeline([
        ('prep', prep),
        ('fs', SelectFromModel(RandomForestClassifier(
            n_estimators=200, random_state=42, n_jobs=-1
        ))),
        ('pca', PCA(n_components=5)),
        ('tree', DecisionTreeClassifier(random_state=42))
    ])
    model_fspca.fit(X_train, y_train)
    pred_fspca = model_fspca.predict(X_test)

    print("\n[FS + PCA]")
    print("Accuracy :", accuracy_score(y_test, pred_fspca))
    print(classification_report(y_test, pred_fspca))

    # Hasil untuk tabel evaluasi
    return {
        "baseline": accuracy_score(y_test, pred_base),
        "fs": accuracy_score(y_test, pred_fs),
        "pca": accuracy_score(y_test, pred_pca),
        "fspca": accuracy_score(y_test, pred_fspca)
    }

hasil1 = run_experiments(df1, 'TIPE_TRANSAKSI', 'Dataset 1 - Transaksi 2021')
hasil1

hasil2 = run_experiments(df2, 'WAVE_CLASS', 'Dataset 2 - Gelombang Laut')
hasil2



 Dataset 1 - Transaksi 2021 

Kategorikal : ['KODE', 'NAMA_PRODUK', 'UNIT', 'TANGGAL', 'NO_TRANSAKSI']
Numerik     : ['QTY_MSK', 'NILAI_MSK', 'QTY_KLR', 'NILAI_KLR']

[BASELINE]
Accuracy : 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     27671

    accuracy                           1.00     27671
   macro avg       1.00      1.00      1.00     27671
weighted avg       1.00      1.00      1.00     27671


[FEATURE SELECTION]
Accuracy : 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     27671

    accuracy                           1.00     27671
   macro avg       1.00      1.00      1.00     27671
weighted avg       1.00      1.00      1.00     27671


[PCA ONLY]
Accuracy : 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     27671

    accuracy                           1.00     27671
   macro avg       1.00      1.00

{'baseline': 1.0,
 'fs': 1.0,
 'pca': 0.9845537757437071,
 'fspca': 0.9914187643020596}

In [ ]:
import pandas as pd

eval_df = pd.DataFrame({
    "Metode": ["Baseline", "Feature Selection", "PCA Only", "FS + PCA"],
    "Dataset 1 Accuracy": [
        hasil1["baseline"],
        hasil1["fs"],
        hasil1["pca"],
        hasil1["fspca"]
    ],
    "Dataset 2 Accuracy": [
        hasil2["baseline"],
        hasil2["fs"],
        hasil2["pca"],
        hasil2["fspca"]
    ]
})

print("\nTabel Evaluasi Sebelum vs Sesudah Preprocessing (Metode Jurnal)\n")
print(eval_df.to_string(index=False))


Tabel Evaluasi Sebelum vs Sesudah Preprocessing (Metode Jurnal)

           Metode  Dataset 1 Accuracy  Dataset 2 Accuracy
         Baseline                 1.0            1.000000
Feature Selection                 1.0            1.000000
         PCA Only                 1.0            0.984554
         FS + PCA                 1.0            0.991419
